In [1]:
import os
%pwd

'/home/mushfiq/Desktop/End-to-End-MLOPS/text_summarizer_end_to_end/research'

In [2]:
os.chdir("../")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: Path
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from src.textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml, create_directories
from src.textsummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH

# basic configuration

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_url=config.source_url,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile

from src.textsummarizer.logging import logger

# components

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not self.config.local_data_file.exists():
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = str(self.config.local_data_file)
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File '{self.config.local_data_file}' already exists!")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Zip file extracted to: {unzip_path}")

In [13]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)

data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2025-07-27 17:31:16,205: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-27 17:31:16,209: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-27 17:31:21,333: INFO: 1534418901: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0CE9:135BC7:B23DAE:CECA3E:68860E05
Accept-Ranges: bytes
Date: Sun, 27 Jul 2025 11:31:18 GMT
Via: 1.1 varnish
X-Served-By: cache-sin-wsss1830049-SIN
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1753615877.478195,VS0,VE634
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: